XGboost全名為eXtreme Gradient Boosting(極限梯度提升)，之所以被稱為Kaggle競賽神器主要是因為在Kaggle上很多比賽的第一名都使用了XGboost。

XGBoost除了精準度很高之外，Training的速度也很異常的快。XGBoost之所以速度快主要是因為整個模型背後是用C++來實作，並且在算法上也做了很多的調整(高度利用CPU多核心平行運算)。


In [2]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\USER\Anaconda3\python.exe -m pip install --upgrade pip' command.


In [3]:
from xgboost import XGBClassifier

In [4]:
import pandas as pd
pd.options.mode.chained_assignment = None

In [112]:
#匯入資料
df_train = pd.read_csv('C:/Users/USER/Desktop/Github/Python Project/train.csv')
df_test =  pd.read_csv('C:/Users/USER/Desktop/Github/Python Project/test.csv')


In [113]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [94]:
#df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [114]:
#資料集 #前處理

x = df_train[['Pclass', 'Age', 'Sex']]
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 3 columns):
Pclass    891 non-null int64
Age       714 non-null float64
Sex       891 non-null object
dtypes: float64(1), int64(1), object(1)
memory usage: 21.0+ KB


In [115]:
#年齡欄位有大量的遺漏值
#簡單用平均值填補
x['Age'] = x['Age'].fillna(x['Age'].mean())
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 3 columns):
Pclass    891 non-null int64
Age       891 non-null float64
Sex       891 non-null object
dtypes: float64(1), int64(1), object(1)
memory usage: 21.0+ KB


In [116]:
y = df_train['Survived']
y

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64

In [118]:
#資料集分割:

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.4, random_state=33)

#x_train
#x_test

,Pclass,Age,Sex
297,1,2.000000,female
71,3,16.000000,female
631,3,51.000000,male
567,3,29.000000,female
21,2,34.000000,male
...,...,...,...
212,3,22.000000,male
674,2,29.699118,male
412,1,33.000000,female
298,1,29.699118,male


In [119]:
#性別欄位轉為數值型態:

x_train['Sex'] = x_train['Sex'].map({'male':0, 'female':1})
x_test['Sex'] = x_test['Sex'].map({'male':0, 'female':1})


In [120]:
x_train

,Pclass,Age,Sex
887,1,19.000000,1
573,3,29.699118,1
862,1,48.000000,1
576,2,34.000000,1
119,3,2.000000,1
...,...,...,...
658,2,23.000000,0
578,3,29.699118,1
728,2,25.000000,0
391,3,21.000000,0


In [121]:
x_test

,Pclass,Age,Sex
297,1,2.000000,1
71,3,16.000000,1
631,3,51.000000,0
567,3,29.000000,1
21,2,34.000000,0
...,...,...,...
212,3,22.000000,0
674,2,29.699118,0
412,1,33.000000,1
298,1,29.699118,0


首先，甚麼參數都不調動，直接丟進隨機森林模型裡看看:


In [122]:
from sklearn.ensemble import RandomForestClassifier

In [123]:
rfc = RandomForestClassifier()

In [124]:
rfc.fit(x_train, y_train)

C:\Users\USER\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [125]:
rfc.score(x_test,y_test)

0.8263305322128851

什麼參數都沒調整的情況下RandomForest的正確率是82.6%

接著，也是甚麼參數都不調動，直接丟進XGBoost模型裡，和隨機森林比較看看:


In [127]:
from xgboost import XGBClassifier

In [128]:
xgbc = XGBClassifier()

In [129]:
xgbc.fit(x_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [130]:
xgbc.score(x_test,y_test)

0.8179271708683473

什麼參數都沒調整的情況下XGBoost的正確率是81.8%